In [11]:
import pandas as pd
import numpy as np
import pickle
import folium
from sklearn.preprocessing import LabelEncoder

In this worksheet, we assemble the waveform data from the OpenEEW AWS server with the earthquake data retrieved from the SSN. Since my data mining process broke a few times, this requires merging a few pickled files.

Also, we assign 'outlier' labels to certain earthquakes due to them occurring near Baja California, far from the the recording devices.

In [126]:
top_1000=pd.read_csv("..\..\data\\top_1000.csv")
top_1000['utc_datetime']=pd.to_datetime(top_1000['utc_datetime'])

In [127]:
devices=pd.read_csv("..\..\data\devices.csv")

In [109]:
top_1000

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,outlier,device_1,device_2,device_4,...,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999,26
0,7.4,15.8033,-96.1337,22.8,"4 km al NORTE de CRUCECITA, OAX",2020-06-23 15:29:03,0.0,"[[0.0021874999999999998, -0.014062499999999999...","[[-0.007187500000000001, -0.018125, -0.024375,...","[[0.0034375, -0.004374999999999999, 0.01000000...",...,"[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[0.007187500000000001, -0.005625000000000001,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",NaN
1,7.2,16.2180,-98.0135,16.0,"14 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-16 23:39:39,0.0,"[[0.09006250000000002, 0.08934375, 0.10725, 0....","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",...,"[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[0.0016562500000000006, 0.01203125, 0.0146562...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",NaN
2,6.5,14.5818,-92.4807,76.2,"37 km al SUROESTE de CD HIDALGO, CHIS",2019-02-01 16:14:12,0.0,"[[-0.0409375, -0.0340625, -0.0234375, -0.03, -...","[[0.0134375, 0.016875, 0.015312500000000001, 0...","[[0.1409375, 0.1465625, 0.15031250000000002, 0...",...,"[[0.27468750000000003, 0.2634375, 0.2684375, 0...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[0.0871875, 0.09312500000000001, 0.0690625, 0...",NaN
3,6.4,13.7792,-93.3382,9.4,"162 km al SUROESTE de CD HIDALGO, CHIS",2019-11-20 04:27:06,0.0,"[[-0.0009375000000000004, -0.00343749999999999...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",...,"[[0.004062500000000001, 0.004375000000000001, ...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[0.004375, 0.00031250000000000006, 0.0203125,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-0.0009375000000000008, 0.0075, 0.0031249999...","[[-0.010624999999999999, -0.00375, -0.0190625,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",NaN
4,6.3,26.6672,-111.0790,16.1,"77 km al NORESTE de LORETO, BCS",2018-01-19 16:17:45,1.0,"[[0.049625, 0.047093750000000004, 0.05315625, ...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",...,"[[-0.05009375, -0.04634375, -0.02784375, -0.02...","[[0.0663125, 0.06659375000000001, 0.07221875, ...","[[0.203, 0.20828124999999997, 0.199375, 0.2068...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...","[[-100000, -100000, -100000, -100000, -100000,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,4.3,14.5795,-93.7258,20.1,"131 km al SUROESTE de MAPASTEPEC, CHIS",2019-02-25 19:38:5

# Stratification/Outliers

In [26]:
m = folium.Map(location=[15.8033, -96.1337],
              zoom_start=5)
for i, quake in top_1000.iterrows():
    folium.Circle(radius=1_000, 
                  location=[quake['latitude'], quake['longitude']], 
                  tooltip="Magnitude: "+ str(quake['magnitude']) + "\n" + "UTC DateTime: " + str(quake['utc_datetime']), 
                  color='orange', fill='orange').add_to(m)

for i, d in devices.iterrows():
        folium.Marker([d['latitude'], d['longitude']],
                      tooltip="Device ID :" + str(d['device_id']), sticky=True).add_to(m)    
m

There's some outliers in our data. Most of the sensors are centered around the southwest where the majority of earthquakes occur, but there's some outliers up by Baja California. I'd like to label those quakes for the purposes of stratification in our train test/split later (to make sure they don't only appear in one set of the data). If the model performs terribly on them, we might just drop them.

I'm labeling quakes by if they're greater than a lattitude of 22.

In [128]:
top_1000['outlier']=top_1000['latitude'].map(lambda x: 1 if x > 22 else 0 )
color_map={0: 'orange', 1: 'green'}

In [25]:
m2 = folium.Map(location=[15.8033, -96.1337],
              zoom_start=7)
for i, quake in top_1000.iterrows():
    lat=quake['latitude']
    long=quake['longitude']
    folium.Circle(radius=1_000, 
                  location=[lat, long], 
                  tooltip="Magnitude: "+ str(quake['magnitude']) + "\n" + "UTC DateTime: " + str(quake['utc_datetime']) + "\n"
                                                                                                + "Lat\Lon: " + f"({lat}, {long})", 
                  color=color_map[quake['outlier']], fill=color_map[quake['outlier']]).add_to(m2)
    
for i, d in devices.iterrows():
        folium.Marker([d['latitude'], d['longitude']],
                      tooltip="Device ID :" + str(d['device_id']), sticky=True).add_to(m2)   
m2

# Assembling data

I had some internet issues and ended up with a handful of different files from scraping data that I need to merge together. Doing it here for cleanliness.

In [3]:
with open('first800.p', 'rb') as f:
    first_800 = pickle.load(f)

In [7]:
len(first_800[0])

800

In [8]:
with open('800-850.p', 'rb') as f:
    next_50 = pickle.load(f)

In [9]:
len(next_50[0])

50

In [27]:
with open('final_150.p', 'rb') as f:
    final_150 = pickle.load(f)

In [29]:
len(final_150[0])

150

In [31]:
final_waveforms=first_800

In [33]:
for i in range(len(final_waveforms)):
    final_waveforms[i].extend(next_50[i])
    final_waveforms[i].extend(final_150[i])

In [36]:
pickle.dump(final_waveforms, open( "final_waveforms.p", "wb" ))

In [130]:
final_waveforms=pickle.load(open("final_waveforms.p", "rb"))

In [131]:
# initialize waveform columns
for device in devices['device_id']:
    col_name="device_"+str(device)
    top_1000[col_name]=np.zeros(top_1000.shape[0])

In [133]:
#add waveforms to df
device_counter = 0
for device in devices['device_id']:
    col_name="device_"+str(device)
    top_1000[col_name]=final_waveforms[device_counter]
    device_counter +=1

In [134]:
top_1000

,magnitude,latitude,longitude,depth,Referencia de localizacion,utc_datetime,outlier,device_1,device_2,device_4,...,device_20,device_21,device_22,device_23,device_24,device_26,device_27,device_29,device_33,device_999
0,7.4,15.8033,-96.1337,22.8,"4 km al NORTE de CRUCECITA, OAX",2020-06-23 15:29:03,0,"[[0.0021874999999999998, -0.014062499999999999...","[[-0.007187500000000001, -0.018125, -0.024375,...","[[0.0034375, -0.004374999999999999, 0.01000000...",...,"[[0.0040625, -0.0021874999999999993, -0.003125...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.007187500000000001, -0.005625000000000001,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,7.2,16.2180,-98.0135,16.0,"14 km al SURESTE de PINOTEPA NACIONAL, OAX",2018-02-16 23:39:39,0,"[[0.09006250000000002, 0.08934375, 0.10725, 0....","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.25896875, 0.25928125, 0.2538125, 0.2502187...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0016562500000000006, 0.01203125, 0.0146562...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,6.5,14.5818,-92.4807,76.2,"37 km al SUROESTE de CD HIDALGO, CHIS",2019-02-01 16:14:12,0,"[[-0.0409375, -0.0340625, -0.0234375, -0.03, -...","[[0.0134375, 0.016875, 0.015312500000000001, 0...","[[0.1409375, 0.1465625, 0.15031250000000002, 0...",...,"[[-0.012187499999999999, -0.0196875, -0.024375...","[[0.27468750000000003, 0.2634375, 0.2684375, 0...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0871875, 0.09312500000000001, 0.0690625, 0..."
3,6.4,13.7792,-93.3382,9.4,"162 km al SUROESTE de CD HIDALGO, CHIS",2019-11-20 04:27:06,0,"[[-0.0009375000000000004, -0.00343749999999999...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.004062500000000001, 0.004375000000000001, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.004375, 0.00031250000000000006, 0.0203125,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-0.0009375000000000008, 0.0075, 0.0031249999...","[[-0.010624999999999999, -0.00375, -0.0190625,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,6.3,26.6672,-111.0790,16.1,"77 km al NORESTE de LORETO, BCS",2018-01-19 16:17:45,1,"[[0.049625, 0.047093750000000004, 0.05315625, ...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",...,"[[-0.006999999999999999, -0.026437500000000003...","[[-0.05009375, -0.04634375, -0.02784375, -0.02...","[[0.0663125, 0.06659375000000001, 0.07221875, ...","[[0.203, 0.20828124999999997, 0.199375, 0.2068...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0

# Processing Data

In [43]:
device_cols=top_1000.columns[top_1000.columns.str.startswith('device')]

Run the cell below to get a count of missing values for a given quake.

In [48]:
null_count = [[np.array_equal(np.zeros((3,300)), arr) for arr in top_1000[device]] for device in device_cols]

not_null_count = []

for i in range(1000):
    x = 0
    for device in null_count:
        x += device[i]
    not_null_count.append(x)
not_null_count

[14,
 13,
 13,
 18,
 12,
 11,
 10,
 13,
 11,
 14,
 10,
 10,
 14,
 9,
 16,
 13,
 13,
 17,
 7,
 12,
 14,
 8,
 16,
 14,
 6,
 14,
 17,
 7,
 12,
 9,
 14,
 12,
 12,
 18,
 20,
 9,
 12,
 13,
 16,
 13,
 7,
 11,
 11,
 13,
 11,
 13,
 18,
 9,
 15,
 10,
 20,
 14,
 15,
 7,
 16,
 11,
 10,
 10,
 13,
 15,
 7,
 10,
 12,
 14,
 12,
 11,
 11,
 14,
 17,
 14,
 12,
 14,
 8,
 16,
 15,
 18,
 12,
 13,
 14,
 15,
 15,
 10,
 18,
 14,
 11,
 22,
 8,
 9,
 8,
 7,
 17,
 7,
 12,
 15,
 13,
 17,
 10,
 12,
 11,
 12,
 17,
 8,
 12,
 16,
 9,
 12,
 18,
 16,
 17,
 11,
 13,
 15,
 14,
 11,
 7,
 15,
 18,
 8,
 16,
 8,
 12,
 10,
 12,
 18,
 15,
 14,
 14,
 16,
 8,
 14,
 12,
 8,
 8,
 11,
 12,
 6,
 16,
 15,
 18,
 15,
 15,
 16,
 11,
 16,
 15,
 12,
 10,
 22,
 12,
 9,
 9,
 17,
 18,
 13,
 17,
 14,
 12,
 16,
 13,
 14,
 11,
 13,
 15,
 16,
 8,
 15,
 15,
 15,
 14,
 9,
 18,
 12,
 14,
 14,
 14,
 13,
 12,
 8,
 12,
 12,
 15,
 13,
 17,
 9,
 15,
 15,
 12,
 8,
 15,
 6,
 11,
 17,
 10,
 17,
 16,
 16,
 13,
 10,
 11,
 12,
 13,
 8,
 16,
 16,
 14,
 7,
 10,
 

Recall that seeing the 0 matrix means that we don't have data for that device for that quake. Each device has at least some quakes it was inactive for, and each quake has at least some devices that weren't recording, and they don't really intersect into a clean set.

Let's also check for incomplete waveforms that might have been recorded. There's missing data throughout many of the .json files scraped.

In [135]:
def count_incomplete(df):
    count = 0
    x,y = df.shape
    for row in range(x):
        for col in range(y):
            i,j = df.iloc[row,col].shape
            if j != 300:
                print((row,col,j))
                count+=1
    return count

In [136]:
count_incomplete(top_1000[device_cols])

(0, 3, 46)
(0, 5, 45)
(0, 6, 31)
(0, 7, 32)
(0, 22, 269)
(1, 10, 0)
(1, 13, 0)
(4, 13, 0)
(7, 10, 0)
(7, 13, 0)
(10, 10, 0)
(10, 13, 0)
(10, 14, 0)
(12, 10, 0)
(12, 13, 0)
(14, 17, 0)
(19, 8, 278)
(19, 9, 295)
(19, 22, 296)
(25, 2, 220)
(25, 22, 244)
(30, 22, 248)
(32, 7, 9)
(32, 17, 0)
(33, 9, 298)
(33, 13, 299)
(37, 8, 0)
(37, 10, 0)
(37, 13, 0)
(39, 3, 0)
(39, 7, 0)
(48, 22, 242)
(51, 22, 293)
(53, 7, 0)
(53, 22, 211)
(54, 17, 0)
(55, 15, 1)
(58, 22, 243)
(62, 4, 299)
(62, 7, 299)
(62, 8, 0)
(62, 10, 0)
(62, 13, 0)
(62, 17, 299)
(64, 10, 0)
(64, 13, 0)
(66, 10, 0)
(66, 13, 0)
(66, 14, 0)
(74, 0, 191)
(80, 3, 0)
(81, 22, 267)
(82, 24, 72)
(88, 16, 50)
(89, 1, 0)
(91, 1, 0)
(91, 2, 0)
(91, 22, 244)
(92, 5, 237)
(94, 10, 0)
(94, 13, 0)
(94, 14, 0)
(98, 13, 0)
(98, 14, 0)
(99, 0, 16)
(99, 10, 0)
(99, 13, 0)
(101, 7, 0)
(104, 2, 0)
(104, 3, 0)
(104, 22, 209)
(105, 17, 0)
(111, 3, 0)
(114, 22, 135)
(115, 10, 0)
(115, 13, 0)
(116, 22, 173)
(117, 2, 285)
(123, 7, 170)
(123, 27, 278)
(124, 1

(865, 3, 137)
(865, 7, 0)
(865, 22, 259)
(866, 1, 0)
(866, 17, 292)
(866, 22, 189)
(869, 22, 79)
(870, 22, 190)
(872, 1, 0)
(874, 22, 168)
(876, 1, 0)
(876, 22, 85)
(881, 1, 0)
(881, 2, 0)
(883, 1, 0)
(883, 2, 0)
(883, 3, 210)
(884, 1, 0)
(884, 2, 0)
(884, 22, 139)
(888, 2, 0)
(901, 16, 215)
(902, 6, 299)
(902, 8, 288)
(902, 9, 299)
(902, 13, 298)
(903, 16, 0)
(903, 22, 292)
(904, 6, 298)
(904, 13, 298)
(904, 22, 188)
(905, 22, 125)
(907, 22, 220)
(908, 3, 0)
(909, 2, 102)
(914, 22, 48)
(916, 3, 0)
(917, 3, 0)
(918, 3, 0)
(920, 3, 0)
(922, 3, 0)
(924, 27, 191)
(926, 16, 264)
(927, 3, 0)
(934, 22, 182)
(936, 17, 0)
(940, 7, 247)
(960, 27, 285)
(961, 22, 287)
(967, 22, 291)
(971, 22, 223)
(976, 17, 0)
(977, 17, 0)
(980, 22, 266)
(990, 7, 0)
(991, 7, 0)
(992, 15, 0)
(993, 27, 134)
(995, 7, 0)
(996, 17, 0)
(996, 22, 275)
(997, 17, 0)


853

We have 853 incomplete waveforms. I don't want to lose their info (for the ones that aren't 0 columns), so let's pad them to length 300 with 0.

In [137]:
def fill_incomplete(df):
    x,y = df.shape
    for row in range(x):
        for col in range(y):
            i,j = df.iloc[row,col].shape
            if j != 300:
                length=300-j
                extension=np.zeros((3, length))
                df.iloc[row,col]=np.append(df.iloc[row,col], extension, axis=1)
    return df

In [138]:
test_fill=fill_incomplete(top_1000[device_cols])

D:\Applications\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
<ipython-input-137-46e570cd0f33>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[row,col]=np.append(df.iloc[row,col], extension, axis=1)


In [139]:
top_1000[device_cols].shape

(1000, 28)

In [140]:
count_incomplete(test_fill)

0

In [141]:
top_1000[device_cols]= test_fill

# State Categories

Post-mortem: After having issues predicting latitude and longitude, I tried to see if I could get any results just predicting the state closest to a given earthquake. Here, we use regular expressions to extract the state from the location reference.

In [19]:
top_1000['Referencia de localizacion']

0                 4 km al NORTE de CRUCECITA, OAX
1      14 km al SURESTE de PINOTEPA NACIONAL, OAX
2           37 km al SUROESTE de CD HIDALGO, CHIS
3          162 km al SUROESTE de CD HIDALGO, CHIS
4                 77 km al NORESTE de LORETO, BCS
                          ...                    
995          78 km al SURESTE de SALINA CRUZ, OAX
996        131 km al SUROESTE de MAPASTEPEC, CHIS
997        160 km al SUROESTE de MAPASTEPEC, CHIS
998    291 km al SUROESTE de PUERTO VALLARTA, JAL
999         93 km al SUROESTE de CD HIDALGO, CHIS
Name: Referencia de localizacion, Length: 1000, dtype: object

In [7]:
top_1000['state']=top_1000['Referencia de localizacion'].str.extract(', ([A-Z].*)')

In [9]:
top_1000['state'].value_counts()

OAX     378
CHIS    335
GRO     105
BCS      37
JAL      30
BC       27
VER      25
MICH     24
COL      13
SIN       9
TAB       6
SON       3
COAH      3
CHIH      2
TAMS      1
SLP       1
DGO       1
Name: state, dtype: int64

Also, let's label encode the states for reference later.

In [12]:
le=LabelEncoder()

In [13]:
top_1000['state_lab']=le.fit_transform(top_1000['state'])

In [25]:
top_1000[['state', 'state_lab']]

,state,state_lab
0,OAX,10
1,OAX,10
2,CHIS,3
3,CHIS,3
4,BCS,1
...,...,...
995,OAX,10
996,CHIS,3
997,CHIS,3
998,JAL,8


In [10]:
pickle.dump(top_1000, open( "top_1000_final.p", "wb" ))

We're finally ready to start modeling. There's some additional reshaping that needs to happen, but I'll leave that for the modelling code because encoding this data as a tensor makes it less readable.